In [ ]:
import branca.colormap as cmp
import folium
import geojson
import geopandas as gpd
import pandas as pd
from shapely import wkb
from tqdm import tqdm

In [ ]:
with open("./../source/spe_geometry.geojson") as f_src:
    sp_geojson = geojson.load(f_src)
sp_geojson.keys()

In [ ]:
mart_df = (
    pd.read_pickle("./../data/forecast_mart.pkl")[["polygon_id", "event_time_id", "posts_count"]]
    .merge(
        pd.read_pickle("./../data/polygons.pkl")[["polygon_id", "polygon_center_wkb", "polygon_wkb"]],
        how="inner",
        on=["polygon_id"],
    )
    .merge(
        pd.read_pickle("./../data/calendar.pkl")[["event_time_id", "year", "month"]],
        how="inner",
        on=["event_time_id"],
    )
)

mart_df = mart_df.groupby(["year", "month", "polygon_id", "polygon_wkb"], as_index=False)[["posts_count"]].sum()
mart_df["event_month"] = mart_df["year"].astype(str) + "-" + mart_df["month"].astype(str).str.zfill(2)

mart_df

In [ ]:
plot_mart_df = gpd.GeoDataFrame(
    mart_df[["polygon_id", "event_month", "posts_count"]],
    geometry=mart_df["polygon_wkb"].apply(wkb.loads),
    crs="EPSG:4326",
)

plot_mart_df

In [ ]:
valid_df = (
    pd.read_pickle("./../data/valid.pkl")
    .rename(columns={"fact_posts_count": "posts_count"})[["polygon_id", "posts_count"]]
    .merge(pd.read_pickle("./../data/polygons.pkl")[["polygon_id", "polygon_wkb"]], how="inner", on=["polygon_id"])
)
valid_df = valid_df.groupby(["polygon_id", "polygon_wkb"], as_index=False)[["posts_count"]].sum()

valid_gdf = gpd.GeoDataFrame(
    valid_df[["polygon_id", "posts_count"]],
    geometry=valid_df["polygon_wkb"].apply(wkb.loads),
    crs="EPSG:4326",
)

valid_gdf

In [ ]:
test_df = (
    pd.read_pickle("./../data/test.pkl")
    .rename(columns={"fact_posts_count": "posts_count"})[["polygon_id", "posts_count"]]
    .merge(pd.read_pickle("./../data/polygons.pkl")[["polygon_id", "polygon_wkb"]], how="inner", on=["polygon_id"])
)
test_df = test_df.groupby(["polygon_id", "polygon_wkb"], as_index=False)[["posts_count"]].sum()

test_gdf = gpd.GeoDataFrame(
    test_df[["polygon_id", "posts_count"]],
    geometry=test_df["polygon_wkb"].apply(wkb.loads),
    crs="EPSG:4326",
)

test_gdf

In [ ]:
vmin = min(plot_mart_df["posts_count"].min(), valid_gdf["posts_count"].min(), test_gdf["posts_count"].min())
vmax = max(plot_mart_df["posts_count"].max(), valid_gdf["posts_count"].max(), test_gdf["posts_count"].max())
colormap = cmp.LinearColormap(colors=["purple", "yellow", "#c7f6b6", "#c7f6b6", "green"][::-1], vmin=vmin, vmax=vmax)

n_steps = 12
list_of_values = plot_mart_df["posts_count"].sort_values()
list_of_values = [i for i in list_of_values if i != 0]
length = len(list_of_values)
index = [list_of_values[int((length / n_steps) * i)] for i in range(n_steps)]
print(index)

# index = [
#     plot_mart_df["posts_count"].min(),
#     plot_mart_df["posts_count"].quantile(0.8),
#     plot_mart_df["posts_count"].quantile(0.99),
#     plot_mart_df["posts_count"].max(),
# ]

colormap = colormap.to_step(index=index)
colormap

In [ ]:
m = folium.Map([59.9375, 30.375], zoom_start=13, tiles=None)

base_map = folium.FeatureGroup(name="osm", overlay=True, control=False)
folium.TileLayer(tiles="OpenStreetMap").add_to(base_map)
base_map.add_to(m)

folium.GeoJson(data=sp_geojson, name="Saint-Petersburg").add_to(m)

style_function = lambda x: {
    "weight": 0.1,
    "color": "black",
    "fillColor": colormap(x["properties"]["posts_count"]),
    "fillOpacity": 0.6,
}
colormap.add_to(m)

for event_month in tqdm(sorted(plot_mart_df["event_month"].unique())):
    if event_month != "2020-02":
        forecast_layer = folium.FeatureGroup(name=event_month, show=False)
    else:
        forecast_layer = folium.FeatureGroup(name="forecast: 2020-02")

    folium.GeoJson(
        plot_mart_df.loc[
            plot_mart_df["event_month"] == event_month, ["polygon_id", "posts_count", "geometry"]
        ].to_json(),
        style_function=style_function,
        tooltip=folium.GeoJsonTooltip(["polygon_id", "posts_count"], labels=True),
    ).add_to(forecast_layer)

    forecast_layer.add_to(m)

valid_layer = folium.FeatureGroup(name="validation: 2020-02")
folium.GeoJson(
    valid_gdf.to_json(),
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(["polygon_id", "posts_count"], labels=True),
).add_to(valid_layer)
valid_layer.add_to(m)

test_layer = folium.FeatureGroup(name="test: 2020-02")
folium.GeoJson(
    test_gdf.to_json(),
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(["polygon_id", "posts_count"], labels=True),
).add_to(test_layer)
test_layer.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m.save("./../maps/map.html")

m